In [6]:
import pandas as pd
import matplotlib.pyplot as plt
from gensim.models.word2vec import Word2Vec
from konlpy.tag import Okt

# 학습시킬 음식 데이터
PATH_NAME = 'C:/Users/gkstk/OneDrive/Desktop/SangMin/Github/Habit-AI/data/'
food_datas = pd.read_csv(PATH_NAME + 'foods.csv')

food_datas[:10]

,id,calcium,carbohydrate,classification,detail_classification,dietary_fiber,energy,fat,food_code,iron,...,name,phosphorus,protein,selenium,serving_size,sodium,vitamin_a,vitamin_b1,vitamin_b2,vitamin_c
0,1,0.00,0.00,튀김류,치킨류,0.00,374.00,0.00,D016426,0.0,...,페퍼간장 치킨,0.00,30.00,0.00,150,824.00,0.00,0.00,0.00,0.00
1,2,0.00,0.00,튀김류,치킨류,0.00,456.00,0.00,D016427,0.0,...,치즈 크리스피 치킨,0.00,34.00,0.00,150,779.00,0.00,0.00,0.00,0.00
2,3,0.00,0.00,튀김류,치킨류,0.00,404.00,0.00,D016428,0.0,...,스윗콤보 치킨,0.00,27.00,0.00,150,698.00,0.00,0.00,0.00,0.00
3,4,0.00,0.00,튀김류,치킨류,0.00,431.00,0.00,D016429,0.0,...,맵고마 치킨,0.00,24.00,0.00,150,756.00,0.00,0.00,0.00,0.00
4,5,0.00,0.00,튀김류,치킨류,0.00,446.00,0.00,D016430,0.0,...,고추깐풍 치킨,0.00,31.00,0.00,150,713.00,0.00,0.00,0.00,0.00
5,6,0.00,0.00,튀김류,치킨류,0.00,387.00,0.00,D016431,0.0,...,순살 쉬림프골드 치킨,0.00,18.00,0.00,150,1082.00,0.00,0.00,0.00,0.00
6,7,0.00,0.00,튀김류,치킨류,0.00,479.00,0.00,D016432,0.0,...,순살닭다리 치킨,0.00,28.00,0.00,150,1169.00,0.00,0.00,0.00,0.00
7,8,0.00,0.00,튀김류,치킨류,0.00,335.00,0.00,D016433,0.0,...,순살화이트 어니언 치킨,0.00,15.00,0.00,150,615.00,0.00,0.00,0.00,0.00
8,9,43.01,42.48,튀김류,치킨류,0.57,543.73,32.44,D018528,0.0,...,간장양념치킨,251.81,20.46,26.17,200,1187.57,12.75,0.00,0.42,0.72
9,10,18.67,14.93,튀김류,치킨류,3.46,468.40,35.42,D018545,0.0,...,치킨너겟,356.13,22.47,32.87,150,843.88,25.82,0.35,0.07,0.00


In [7]:
# 데이터 프레임 내에 빈 값이 있는지 확인
print(food_datas.isnull().values.any())

False


In [8]:
rows_with_na = food_datas[food_datas.isna().any(axis=1)]
print(rows_with_na)

Empty DataFrame
Columns: [id, calcium, carbohydrate, classification, detail_classification, dietary_fiber, energy, fat, food_code, iron, moisture, name, phosphorus, protein, selenium, serving_size, sodium, vitamin_a, vitamin_b1, vitamin_b2, vitamin_c]
Index: []

[0 rows x 21 columns]


In [10]:
# 한국어만 남김
food_datas['newName'] = food_datas['detail_classification'] + food_datas['name']
food_datas['newName'] = food_datas['newName'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", " ", regex=True)
food_datas['newName'].head(10)

,NO,식품명
0,1,메밀전병
1,2,수수부꾸미
2,3,약식
3,4,경단 깨
4,5,경단 콩
5,6,기피편
6,7,녹두시루떡
7,8,무지개떡
8,9,송편 깨
9,10,송편 콩


In [11]:
# 불용어 리스트
stop_words = ['가', '걍', '것', '고', '과', '는', '도', '들', '등', '때', '로', '를', '뿐', '수', '아니', '않', '없', '에', '에게', '와', '으로', '은', '의', '이', '이다', '있', '자', '잘', '좀', '하다', '한', '조각', '개', '것', '대', '소' ,'단계', '등급', '포함', '미니', '개입']

In [12]:
okt = Okt()
token_list = []

for food in food_datas['식품명']:
    # Okt 형태소 분석기를 사용하여 토큰화
    token = okt.morphs(food, stem=True)
    # 불용어 제거
    token = [word for word in token if not word in stop_words]
    token_list.append(token)
    
token_list[:10]

[['메밀', '전병'],
 ['수수', '부꾸미'],
 ['약식'],
 ['경단', '깨'],
 ['경단', '콩'],
 ['기피', '편'],
 ['녹두', '시루떡'],
 ['무지개떡'],
 ['송편', '깨'],
 ['송편', '콩']]

In [13]:
# Word2Vector로 모델 학습
model = Word2Vec(token_list, vector_size=100, window=5, min_count=1, sg=1, workers=4)

vector = model.wv['기피']
model.wv.vectors.shape

(3710, 100)

In [14]:
# model.wv.save('한국어_음식모델_한상민.kv')